In [7]:
%load_ext autoreload
%autoreload 2

import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from pyproj import Proj, CRS, Transformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from sbp_tools.sbp_track_processing import process_track, save_track_to_radex, save_track, read_segypos

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm35n, always_xy=True)

data_path = r'D:\SBP_SES\AI-68_converted\FZ_stg2_GEOCOORDS_ALL'
# track
save_to = r'D:\SBP_SES\AI-68_Conv_ProcPos\SEGY_DATA_TRACK_FZ_STG2_LF1.txt'

is_utm = False
is_arcsec = True
coord_scaler = 1
smooth = False
year_check = False
year = 2024

#pos_files = fes.file_ext_search('.rawpos', data_path, recursive=True)
#print(len(pos_files))
pos_files = fes.file_ext_search('.txt', data_path, recursive=True)
print(len(pos_files))

pos_files_LF = []

for file in pos_files:
    if 'LF1' in file:
        pos_files_LF.append(file)
    
    

segy_pos_objs = []
bad_data_dict = {}
fine_data_dict = {}

read_segypos(pos_files_LF, fine_data_dict, bad_data_dict, segy_pos_objs, year, utm_coords = is_utm, arcsecs=is_arcsec, coord_scale=coord_scaler)
process_track(segy_pos_objs, transformer, window_length=201, smooth=smooth, utm_coords=is_utm)
save_track(segy_pos_objs, save_to, smoothed=smooth)
#save_track_to_radex(segy_pos_objs)

Searching *.txt files in directory:D:\SBP_SES\AI-68_converted\FZ_stg2_GEOCOORDS_ALL
4776
File AI6820240903_161854_MF_RAW_LF1_rawpos is done 1 of 796
File AI6820240903_162037_MF_RAW_LF1_rawpos is done 2 of 796
File AI6820240903_163537_MF_RAW_LF1_rawpos is done 3 of 796
File AI6820240903_165038_MF_RAW_LF1_rawpos is done 4 of 796
File AI6820240903_170539_MF_RAW_LF1_rawpos is done 5 of 796
File AI6820240903_171616_MF_RAW_LF1_rawpos is done 6 of 796
File AI6820240903_173117_MF_RAW_LF1_rawpos is done 7 of 796
File AI6820240903_174543_MF_RAW_LF1_rawpos is done 8 of 796
File AI6820240903_183536_MF_RAW_LF1_rawpos is done 9 of 796
File AI6820240903_185037_MF_RAW_LF1_rawpos is done 10 of 796
File AI6820240903_190537_MF_RAW_LF1_rawpos is done 11 of 796
File AI6820240903_192038_MF_RAW_LF1_rawpos is done 12 of 796
File AI6820240903_193539_MF_RAW_LF1_rawpos is done 13 of 796
File AI6820240903_195040_MF_RAW_LF1_rawpos is done 14 of 796
File AI6820240903_200541_MF_RAW_LF1_rawpos is done 15 of 796
File 

In [3]:
from shutil import move

move_to_bad_data = r'C:\Data\ABP56_FZ_UTM35N\sgy_ps3_SLF_FIXED_Bad'


for file in bad_data_dict.keys():
    segy_name = file[:-7]
    sgy_path = os.path.join(data_path, segy_name + '.sgy')
    pos_path = os.path.join(data_path,file + '.txt')
    
    move(sgy_path, move_to_bad_data)
    move(pos_path, move_to_bad_data)